tip. 손실함수는 고정한다. 경사하강법을 통해 바뀌는 것은 **예측함수의 파라미터**이다.

cf) numpy에서 `:`만 존재하면 모든 행을 의미


## 3.5. 데이터 전처리

np로 데이터를 불러온 후 x, y로 분리해준다.
이후 x, y의 데이터를 정규화해준다. (하나의 방법은 각각의 평균으로 빼주는 것)

## 3.6. 예측 계산

예측 계산을 위해서는 예측 함수(pred)를 정의해야 한다.

머신러닝이란 결국 예측 함수를 가능한 한 정확하게 만드는 것이기 때문에 예측 함수의 기준이 되는 변수(parameter)를 설정해주어야 한다.

일차함수인 단일선형회귀를 예로 들면 Weight, Bias를 설정해준다.

```python
W = torch.tensor(1.0, requires_grad=True, dtype=torch.float32)
B = torch.tensor(1.0, requires_grad=True, dtype=torch.float32)
```

초깃값은 모두 W=1, B=1로 설정해준다.

이때 예측함수는 다음과 같이 정의할 수 있다: `Yp = W * X + B`

## 3.7. 손실 계산

손실 함수는 예측함수를 통해 계산된 값과 실제 값의 차이를 계산해주는 함수이다.

여러가지 알고리즘이 있으나 가장 대표적인 손실 함수는 평균 제곱 오차(MSE)임.

```python
def mse(Yp, Y):
    return ((Yp - Y) ** 2).mean()
```

이 함수에 실제로 Yp, Y를 대입하면 손실값을 구할 수 있다.


## 3.8. 경사 계산

예측 계산과 손실 계산을 통해 구한 손실의 경사를 통해 예측 함수의 파라미터를 수정해준다.
이를 위해 편미분을 사용해야 하고 손실을 구하는 과정이 복잡할수록 미분을 하는 과정도 복잡하지만 이 지점에서 PyTorch의 Define By Run 이 큰 장점을 갖는다:

```python
loss.backward()
```

이러면 자동으로 손실의 경사가 예측함수의 파라미터에 어떤 영향을 미치는지 계산(`W.grad`, `B.grad`)해줌

이후 파라미터(W,B)의 값을 갱신하기 위해 기존 값에서 학습률(lr)과 경사값을 곱해준다.

이때 주의할 점은 W와 B는 requires_grad=True이기 때문에 그냥 값을 수정하면 오류가 발생한다. 

이를 해결하기 위해서는 `with torch.no_grad()`를 사용해야 한다.

```python
with torch.no_grad():
    W -= lr * W.grad
    B -= lr * B.grad
```

이후 PyTorch에서 경사값은 누적되기 때문에 초기화해주어야 한다.

```python
W.grad.zero_()
B.grad.zero_()
```

---

그러나 실제 딥러닝에서는 W, B를 직접 수정하기 보다는 최적화함수(Optimizer)를 사용함

가장 단순하고 대표적인 최적화함수는 SGD(Stochastic Gradient Descent)임

```python
import torch.optim as optim

optimizer = optim.SGD([W, B], lr=0.01)
```

이후 반복 계산할 때

```python
optimizer.step() # 자동으로 W, B 갱신
optimizer.zero_grad() # 경사값 초기화
```

결과를 확인해보면 수동으로 W,B를 갱신한 것과 전혀 차이가 없음을 확인할 수 있다.

이후는 예측 계산, 손실 계산, 경사 계산의 반복이다

cf) 학습 기록에 관햬:

```python
if epoch % 10 == 0:
    item = np.array([epoch, loss.item()])
    history = np.vstack((history, item))
    print(f"Epoch {epoch} : Loss {loss:.4f}")
```

1. epoch는 반복 횟수를 말함
2. 각 반복횟수는 손실값과 함께 저장
3. vstack 은 vertical stack인데 두 개의 배열을 세로로 누적해서 쌓는 것을 말함 -> 나중에 그래프로 표현할 때 쓰기 위해 history에 저장